In [1]:
import os
os.chdir(os.getenv("NB_HOME"))

In [2]:
!make create_tables

python table_creation/create_tables.py


# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
import os
import glob
import psycopg2
import pandas as pd
import json

In [4]:
from table_creation.create_tables import get_connector_and_cursor, generate_uri
conn, cur = get_connector_and_cursor(override_env={"DATABASE_NAME":"sparkifydb"})

from sqlalchemy import create_engine

from scripts.templating import choose_template

generate_uri(template="postgres_url", override_env={"DATABASE_NAME":"sparkifydb"})
engine = create_engine(generate_uri(template="postgres_url", override_env={"DATABASE_NAME":"sparkifydb"}))

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [6]:
def filter_out_val(df, col_name, val):
    """
    - Receives DataFrame, column and value to be filter out
    - Returns a DataFrame
    """
    return df[df[col_name] != val]

def filter_out(df, filter_out_dict):
    """
    - Receives a DataFrame and a dict
    - uses the dict and for each key filter out that value
    -- filter_out_multiple_vals haven't been implemented yet (https://github.com/pandas-dev/pandas/pull/29636/)
    - Returns DataFrame after filters
    """
    df = df.copy()
    for col_name, val in filter_out_dict.items():
        tmp_df = filter_out_val(df, col_name, val)
        df = tmp_df
    return df


def df_cleanse(df, filter_cols, filter_out_dict={}, astype={}, rename_map=[], duplicates_considering=[]):
    """
    - Since Pandas hasn't implemented UPSERT yet, this method considers also this adjustment
    - It runs all cleanse in a dataframe before injection in a table
    - Returns DataFrame
    """
    
    df = df[filter_cols]
    df = filter_out(df, filter_out_dict)
    df = df.astype(astype)
    df = df.rename(columns=rename_map)
    if len(duplicates_considering) > 0:
        df = df.drop_duplicates(duplicates_considering)
    else:
        df = df.drop_duplicates()
    return df

In [7]:
def to_str(val):
    return str(tuple(val)).replace("'","")

def prepare_columns(df, template_formatted="%({})s"):
    df_columns= df.columns
    flatten= []
    for elem in df_columns:
        flatten.append(template_formatted.format(elem))
    
    return to_str(df_columns), to_str(flatten)


def inject_template(df
        , database_name: str
        , conflict_rule: str =""
        , conflict_action="NOTHING"
        , template_name="insert_data"):
    """
    Requires
    df :
        pandas dataframe
    
    database_name :
        Name of SQL table.
    
    conn : SQLAlchemy connectable(engine/connection) or database string URI
        or sqlite3 DBAPI2 connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library.
        If a DBAPI2 object, only sqlite3 is supported.
    
    """
    
    template = choose_template(template_name)
    columns_names, columns_names_formatted = prepare_columns(df)
    
    d={}
    d["columns_names"] = columns_names
    d["columns_names_formatted"] = columns_names_formatted
    d["database_name"]= database_name
    d["conflict_rule"] = conflict_rule
    d["conflict_action"] = conflict_action
    
    return template.format(**d)

def inject_data(
        df
        , cur
        , conn
        , injection_rule
        , show_status_at = 250
    ):
    """
    Requires
    df
        pandas dataframe
        
    conn : SQLAlchemy connectable(engine/connection) or database string URI
        or sqlite3 DBAPI2 connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library.
        If a DBAPI2 object, only sqlite3 is supported.
    
    """
    
    
    injection_data_list = json.loads( df.to_json(orient="records") )
        
    for counter, injection_data in enumerate(injection_data_list):
        letter =""
        size = len(injection_data_list) -1
        
        cur.execute(injection_rule, injection_data)
        conn.commit()

        if counter == len(injection_data_list) -1:
            letter = "! DONE"
        else: 
            if size < show_status_at:
                letter = "."
            else:
                if counter % show_status_at == 0:
                    letter = "."
        
        print(letter, end="")

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
song_files = get_files("data/song_data")

In [9]:
filepath = song_files[0]
filepath

'/home/jovyan/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [10]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [11]:
def read_json(file):
    return pd.read_json(file, lines=True)

df = pd.concat(
    map(
        read_json
        , song_files
    )
)

df.shape

(71, 10)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [12]:
songs_desired_columns= [
    "song_id"
    , "title"
    , "artist_id"
    , "year"
    , "duration"
]

display(
    df[songs_desired_columns]\
    .drop_duplicates()\
    .head()
)

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
0,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
0,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
0,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
inject_data(
        df[songs_desired_columns]
        , cur
        , conn
        , injection_rule= inject_template(
                                            df[songs_desired_columns]
                                            , "songs"
                                            , conflict_rule="(song_id)"
                                            , conflict_action="""NOTHING"""
                                        )
    )

......................................................................! DONE

In [14]:
## Even though this works, the solution above let us use several runs (due the conflict rules and action), so its being elected despite this one
# df[songs_desired_columns]\
#     .drop_duplicates()\
#     .to_sql('songs', con=engine, if_exists='append',index=False)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
artists_desired_columns= [
    "artist_id"
    , "artist_name"
    , "artist_location"
    , "artist_latitude"
    , "artist_longitude"
]

artists_rename_map = {
    "artist_name"        : "name"
    , "artist_location"  : "location"
    , "artist_latitude"  : "latitude"
    , "artist_longitude" : "longitude"
}


display(df[artists_desired_columns].head())

display(
        df[artists_desired_columns]\
        .rename(columns=artists_rename_map)\
        .drop_duplicates()\
        .head())



,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
0,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
0,ARKRRTF1187B9984DA,Sonora Santanera,,NaN,NaN
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
0,ARXR32B1187FB57099,Gob,,NaN,NaN


,artist_id,name,location,latitude,longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
0,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
0,ARKRRTF1187B9984DA,Sonora Santanera,,NaN,NaN
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
0,ARXR32B1187FB57099,Gob,,NaN,NaN


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [16]:
inject_data(
        df[artists_desired_columns].rename(columns=artists_rename_map)
        , cur
        , conn
        , injection_rule= inject_template(
                                            df[artists_desired_columns].rename(columns=artists_rename_map)
                                            , "artists"
                                            , conflict_rule="(artist_id)"
                                            , conflict_action="""NOTHING"""
                                        )
    )

......................................................................! DONE

In [17]:
# df[artists_desired_columns]\
# .rename(columns=artists_rename_map)\
# .drop_duplicates()\
# .to_sql('artists', con=engine, if_exists='append',index=False)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files("data/log_data")

In [19]:
log_filepath = log_files[0]

In [20]:
log_df = pd.read_json(log_filepath, lines=True)
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [21]:
log_df = pd.concat(
    map(
        read_json
        , log_files
    )
)

log_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
386,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
log_df['time'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).astype(str)
log_df['hour'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.hour.astype(str)
log_df['day'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.day.astype(str)

## warning of deprecation
# log_df['week'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.week.astype(str)

log_df['week'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.isocalendar().week.astype(str)
log_df['month'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.month.astype(str)
log_df['year'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.year.astype(str)
log_df['weekday'] = pd.to_datetime(log_df['ts'], unit='ms', utc=True).dt.weekday.astype(str)


In [23]:
time_df = log_df[log_df["page"] == "NextSong"].copy()
time_df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,time,hour,day,week,month,year,weekday
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:01:46.796000+00:00,21,1,44,11,2018,3
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:05:52.796000+00:00,21,1,44,11,2018,3
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:08:16.796000+00:00,21,1,44,11,2018,3


In [24]:
times_desired_columns = [
    "time", "hour", "day", "week", "month", "year", "weekday"
]


time_rename_map = {
    "time"               : "start_time"
}

times_df = df_cleanse(
    df=time_df
    , filter_cols=times_desired_columns
    # , filter_out_dict= {"userId":""}
    # , astype= {"userId": int}
    , rename_map= time_rename_map
    # , duplicates_considering=["start_time"]
)

display(times_df)

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796000+00:00,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796000+00:00,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796000+00:00,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796000+00:00,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796000+00:00,21,1,44,11,2018,3
...,...,...,...,...,...,...,...
382,2018-11-30 18:40:05.796000+00:00,18,30,48,11,2018,4
383,2018-11-30 18:44:36.796000+00:00,18,30,48,11,2018,4
384,2018-11-30 18:47:58.796000+00:00,18,30,48,11,2018,4
385,2018-11-30 18:51:24.796000+00:00,18,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
inject_data(
        times_df
        , cur
        , conn
        , injection_rule= inject_template(
                                            times_df
                                            , "time"
                                            , conflict_rule="(start_time)"
                                            , conflict_action="""NOTHING"""
                                        )
    )

............................! DONE

In [26]:
# times_df\
# .to_sql('time', con=engine, if_exists='append',index=False)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [27]:
users_desired_columns= [
    "userId"
    , "firstName"
    , "lastName"
    , "gender"
    , "level"
    
]

users_rename_map = {
    "userId" : "user_id"
    , "firstName"        : "first_name"
    , "lastName"         : "last_name"
}


In [28]:
users_df = df_cleanse(
    df=log_df
    , filter_cols=users_desired_columns
    , filter_out_dict= {"userId":""}
    , astype= {"userId": int}
    , rename_map= users_rename_map
    # , duplicates_considering=["user_id"]
)

display(users_df)

,user_id,first_name,last_name,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
11,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free
...,...,...,...,...,...
13,21,Preston,Sanders,M,free
3,38,Gianna,Jones,F,free
35,5,Elijah,Davis,M,free
62,82,Avery,Martinez,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [29]:
inject_data(
        users_df
        , cur
        , conn
        , injection_rule= inject_template(
                                            users_df
                                            , "users"
                                            , conflict_rule="(user_id)"
                                            , conflict_action=""" UPDATE SET level = EXCLUDED.level"""
                                        )
    )

........................................................................................................! DONE

In [30]:
# users_df\
# .to_sql('users', con=engine, if_exists='append',index=False)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [31]:
songplays_desired_columns = [
    "time",
    "userId",
    "level",
    # songid,
    # artistid,
    "sessionId",
    "location",
    "userAgent",
    "song",
    "artist",
    "length"
]
songplays_df = df_cleanse(
    df=log_df
    , filter_cols=songplays_desired_columns
    # , filter_out_dict= {"userId":""}
    # , astype= {"userId": int}
    , rename_map= {}
    # , duplicates_considering=["start_time"]
).reset_index(drop=True)

songplays_df

,time,userId,level,sessionId,location,userAgent,song,artist,length
0,2018-11-01 20:57:10.796000+00:00,39,free,38,"San Francisco-Oakland-Hayward, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,NaN
1,2018-11-01 21:01:46.796000+00:00,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,NaN
2,2018-11-01 21:01:46.796000+00:00,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",You Gotta Be,Des'ree,246.30812
3,2018-11-01 21:02:12.796000+00:00,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,NaN
4,2018-11-01 21:05:52.796000+00:00,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Flat 55,Mr Oizo,144.03873
...,...,...,...,...,...,...,...,...,...
8047,2018-11-30 18:44:36.796000+00:00,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Besos De Ceniza,Timbiriche,202.60526
8048,2018-11-30 18:47:58.796000+00:00,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Rose,A Perfect Circle,206.05342
8049,2018-11-30 18:51:24.796000+00:00,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",The Haunting,Anberlin,348.68200
8050,2018-11-30 18:53:13.796000+00:00,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,NaN


In [32]:
song_select="""
select
    song_id, artist_id
from 
    songs
inner join artists using (artist_id)
where
    songs.title=%s
    and artists.name=%s
    and songs.duration=%s
"""


In [33]:
songplay_table_insert= """
    Insert INTO songplays (
        start_time,
        user_id,
        level,
        song_id,
        artist_id,
        session_id,
        location,
        user_agent 
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)

"""

In [34]:
show_status_at = 250

for index, row in songplays_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    if songid != artistid:
        print(songid, artistid)

    # insert songplay record
    songplay_data = (
        #check what to use here, maybe index? using now auto_incremental approach. a.k.a. serial
        row.time,
        row.userId,
        row.level,
        songid,
        artistid,
        row.sessionId,
        row.location,
        row.userAgent
    )
    
    if row.userId == '':
        pass
    else:
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()

    if index % show_status_at == 0:
        print(index, songplay_data)


0 ('2018-11-01 20:57:10.796000+00:00', 39, 'free', None, None, 38, 'San Francisco-Oakland-Hayward, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
250 ('2018-11-03 18:03:23.796000+00:00', '95', 'paid', None, None, 152, 'Winston-Salem, NC', '"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"')
500 ('2018-11-05 02:21:55.796000+00:00', '44', 'paid', None, None, 237, 'Waterloo-Cedar Falls, IA', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0')
750 ('2018-11-05 16:04:05.796000+00:00', '97', 'paid', None, None, 147, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
1000 ('2018-11-06 19:30:29.796000+00:00', '97', 'paid', None, None, 293, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) Appl

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [35]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.